# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
import re
import pandas as pd
import numpy as np
#from sqlalchemy import create_engine
import sqlite3

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#from sklearn.compose import ColumnTransformer
import pickle


In [26]:
pd.__version__

'1.3.5'

In [27]:
import sklearn
sklearn.show_versions()


System:
    python: 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]
executable: c:\users\hedey\appdata\local\programs\python\python37\python.exe
   machine: Windows-10-10.0.19041-SP0

Python dependencies:
          pip: 23.1.2
   setuptools: 47.1.0
      sklearn: 1.0.2
        numpy: 1.21.6
        scipy: 1.7.3
       Cython: None
       pandas: 1.3.5
   matplotlib: 3.5.1
       joblib: 1.1.0
threadpoolctl: 3.1.0

Built with OpenMP: True


In [28]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hedey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hedey\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
# load data from database
#engine = create_engine('sqlite:///DisasterResponse.db')
#df = pd.read_sql_table('MessageCategory', engine)

# connect to the database
conn = sqlite3.connect('DisasterResponse.db')

# run a query
df=pd.read_sql('SELECT * FROM MessageCategory', conn)
df=df[df['related']!=2]
#df['child_alone'].iloc[0]=1
X = df[['message', 'genre']]#.values
#X = df[['message']]#.values
Y = df.iloc[:,4:]#.values
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
Y.shape

(26028, 36)

In [31]:
X.shape

(26028, 2)

In [32]:
X['message'].shape

(26028,)

In [33]:
X['genre'].shape

(26028,)

In [34]:
df=df[df['related']!=2]

In [35]:
Y.shape

(26028, 36)

In [36]:
single_value_targets=[]
target_cols=list(df.columns[4:])
for col in target_cols:#df.iloc[:,4:].columns:
    if len(df[col].unique())==1:
        print(col, df[col].unique(), len(df[col].unique()), target_cols.index(col))
        single_value_targets.append(target_cols.index(col))

child_alone [0] 1 9


In [37]:
df.child_alone.unique()

array([0], dtype=int64)

In [38]:
df.related.unique()

array([1, 0], dtype=int64)

In [39]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [40]:
target_names=list(df.columns[4:])

### 2. Write a tokenization function to process your text data

In [41]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        if self.key=='genre':
            return data_dict[[self.key]]
        else:
            return data_dict[self.key]


In [43]:
message_transformer = Pipeline([
    ('selector', ItemSelector(key='message')),
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tdfidf', TfidfTransformer())])

In [44]:
genre_transformer = Pipeline([
    ('selector', ItemSelector(key='genre')),
    ('onehot', OneHotEncoder())])

In [55]:
msg_gnre_pipeline=Pipeline([
    ('features', FeatureUnion([
        ('message_pipe', message_transformer),
        ('genre_pipe', genre_transformer)
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(verbose=4)))
    #('clf', MultiOutputClassifier(LogisticRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [57]:
for col in y_train:
    print(y_train[col].unique())

[1 0]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


In [58]:
X_train.head()

,message,genre
15445,The failure of rescuers to reach these areas p...,news
15974,"NEW YORK, 21 September 2007 - The exceptionall...",news
7112,Human rights group Be a pillar on which those ...,direct
3285,Please send us information concerning them.,direct
23842,In deference to the concerns of the Malian gov...,news


In [59]:
y_train.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
15445,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15974,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
7112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3285,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23842,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
for i in single_value_targets:
    y_train.iloc[0,i]=1

In [61]:
y_train.iloc[0,9]

1

In [63]:
msg_gnre_pipeline.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s


...

building tree 3 of 100


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s remaining:    0.0s


building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 10

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   28.4s finished


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('message_pipe',
                                                 Pipeline(steps=[('selector',
                                                                  ItemSelector(key='message')),
                                                                 ('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001AD8AB20D38>)),
                                                                 ('tdfidf',
                                                                  TfidfTransformer())])),
                                                ('genre_pipe',
                                                 Pipeline(steps=[('selector',
                                                                  ItemSelector(key='genre')),
                                                                 ('onehot',
                                            

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [64]:
#target_names=list(df.columns[4:])

y_pred_msg_gnre_rfc = msg_gnre_pipeline.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remain

In [29]:
pd.DataFrame(y_pred_msg_gnre_rfc).sum(axis=0)

0     4742
1      508
2        0
3     1708
4       23
5       25
6        8
7        1
8       11
9        0
10     101
11     297
12     126
13      10
14       7
15       1
16       4
17      32
18      15
19       3
20      30
21      20
22       2
23       0
24       0
25       0
26       0
27       2
28    1038
29     174
30     198
31       1
32     380
33       4
34       8
35     475
dtype: int64

In [30]:
y_test.values.shape

(5206, 36)

In [31]:
y_pred_msg_gnre_rfc.shape

(5206, 36)

In [65]:
report_msg_gnre_rfc = classification_report(y_test.values, y_pred_msg_gnre_rfc, target_names=target_names)
print(report_msg_gnre_rfc)

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      3959
               request       0.87      0.49      0.62       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.78      0.61      0.69      2156
          medical_help       0.62      0.05      0.09       431
      medical_products       0.78      0.07      0.13       264
     search_and_rescue       0.64      0.05      0.09       151
              security       0.50      0.01      0.02       106
              military       0.78      0.04      0.08       175
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.25      0.39       344
                  food       0.88      0.43      0.58       586
               shelter       0.90      0.24      0.38       487
              clothing       0.73      0.10      0.18        79
                 money       0.80      

c:\users\hedey\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\hedey\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\hedey\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
parameters = {
    'features__message_pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4]
}

# create grid search object
cv = GridSearchCV(msg_gnre_pipeline, parameters, verbose=4)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, features__message_pipe__vect__ngram_range=(1, 1);, score=0.236 total time= 2.8min


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# predict on test data
y_pred_cv = cv.predict(X_test)

report_cv = classification_report(y_test, y_pred_cv, target_names=target_names)
print(report_cv)
#print(classification_report(np.hstack(y_test), np.hstack(y_pred)))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [67]:
filename = "msg_gnre_pipeline.pkl" # save model
pickle.dump(msg_gnre_pipeline, open(filename, "wb"))

In [66]:
filename = "cv_model.pkl" # save model
pickle.dump(cv, open(filename, "wb"))

NameError: name 'cv' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.